# Build custom container for Vertex Prediction

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/02-perarm-features-bandit


In [2]:
# PREFIX = 'mabv1'
VERSION        = "v2"                       # TODO
PREFIX         = f'rec-bandits-{VERSION}'   # TODO

print(f"PREFIX: {PREFIX}")

PREFIX: rec-bandits-v2


In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "rec-bandits-v2"
VERSION                  = "v2"

BUCKET_NAME              = "rec-bandits-v2-hybrid-vertex-bucket"
BUCKET_URI               = "gs://rec-bandits-v2-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://rec-bandits-v2-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"
DATA_PATH_KFP_DEMO       = "gs://rec-bandits-v2-hybrid-vertex-bucket/data/kfp_demo_data/u.data"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_NAME    = "mvlens_rec_bandits_v2"
BIGQUERY_TABLE_NA

In [4]:
# TODO incorporate to 00-env-setup
DOCKERNAME_02_PRED = 'Dockerfile_predict_mab_02'
IMAGE_NAME_02_PRED = "pred-perarm-feats-03"
IMAGE_URI_02_PRED  = f"gcr.io/hybrid-vertex/{IMAGE_NAME_02_PRED}"

print(f"DOCKERNAME_02_PRED = {DOCKERNAME_02_PRED}")
print(f"REPOSITORY         = {REPOSITORY}")
print(f"IMAGE_NAME_02_PRED = {IMAGE_NAME_02_PRED}")
print(f"IMAGE_URI_02_PRED  = {IMAGE_URI_02_PRED}")

DOCKERNAME_02_PRED = Dockerfile_predict_mab_02
REPOSITORY         = rl-movielens-rec-bandits-v2
IMAGE_NAME_02_PRED = pred-perarm-feats-03
IMAGE_URI_02_PRED  = gcr.io/hybrid-vertex/pred-perarm-feats-03


## Create Artifact Repository
If you don't have an existing artifact repository, create one using the gcloud command below

In [5]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

In [6]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

## Set gcloudignore

In [17]:
! gcloud config set gcloudignore/enabled true

Updated property [gcloudignore/enabled].


run this in terminal from root to clear `__pycache__` files...

In [ ]:
# find . | grep -E "(/__pycache__$|\.pyc$|\.pyo$)" | xargs rm -rf

overwrite `.gcloudignore`...

In [20]:
%%writefile .gcloudignore
.gcloudignore
/WIP/
/imgs/*
learning/*
*/__pycache__/*
*.pkl
*.png
*.ipynb
.git
.github
.gitignore
.DS_Store
.ipynb_checkpoints/*
*cpython-37.pyc
**.cpython-310.pyc
README.md
*.tfrecord
src/ranking/*
src/archive/*
04-pipelines/*
01-baseline-perarm-bandit/*

Overwriting .gcloudignore


check eligble files...

In [21]:
!gcloud meta list-files-for-upload

Dockerfile_perarm_feats
requirements.txt
Dockerfile_train_my_perarm_env
Dockerfile_predict_mab_02
Dockerfile_perarm_feats_tpu
cloudbuild.yaml
src/per_arm_rl/utils_config.py
src/per_arm_rl/perarm_task.py
src/per_arm_rl/__init__.py
src/per_arm_rl/my_per_arm_py_env.py
src/per_arm_rl/policy_util.py
src/per_arm_rl/train_utils.py
src/per_arm_rl/trainer_baseline.py
src/per_arm_rl/data_utils.py
src/per_arm_rl/data_config.py
src/prediction/requirements.txt
src/prediction/__init__.py
src/prediction/prestart.sh
src/prediction/main.py
src/prediction/data_config.py
src/perarm_features/train_perarm.py
src/perarm_features/reward_factory.py
src/perarm_features/emb_features.py
src/perarm_features/agent_factory.py
src/perarm_features/__init__.py
src/perarm_features/trainer_common.py
src/perarm_features/task.py
src/perarm_features/eval_perarm.py
src/perarm_features/ranking_bandit_policy.py


In [22]:
!tree src -L 2 -I "archive"

src
├── per_arm_rl
│   ├── __init__.py
│   ├── data_config.py
│   ├── data_utils.py
│   ├── my_per_arm_py_env.py
│   ├── perarm_task.py
│   ├── policy_util.py
│   ├── train_utils.py
│   ├── trainer_baseline.py
│   └── utils_config.py
├── perarm_features
│   ├── __init__.py
│   ├── agent_factory.py
│   ├── emb_features.py
│   ├── eval_perarm.py
│   ├── ranking_bandit_policy.py
│   ├── reward_factory.py
│   ├── task.py
│   ├── train_perarm.py
│   └── trainer_common.py
├── prediction
│   ├── __init__.py
│   ├── data_config.py
│   ├── main.py
│   ├── prestart.sh
│   └── requirements.txt
└── ranking
    ├── jw_perarm_mv_env.py
    ├── jw_perarm_mv_env_tf.py
    ├── ranking_trainer.py
    ├── repeated_feature_perarm_network.py
    ├── stationary_stochastic_repeated_feature_py_environment.py
    └── tmp_cascading_bandit_perarm_env.py

4 directories, 29 files


## Build Dockerfile

In [24]:
# COPY src/prediction /app
# COPY src/perarm_features /app/perarm_features

# CMD ["sh", "-c", "uvicorn src.prediction.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

# ENV PYTHONUNBUFFERED True

# ARG FOO=bar
# ARG FOO
# RUN echo "these steps will be executed again if the value of FOO changes"

In [25]:
dockerfile = f'''
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.10

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl
COPY src/prediction $APP_HOME/src/prediction

RUN ls

RUN pip3 install --upgrade pip
RUN pip3 install --no-cache-dir -r $APP_HOME/src/prediction/requirements.txt

CMD ["sh", "-c", "uvicorn src.prediction.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]

'''
print(dockerfile)


FROM tiangolo/uvicorn-gunicorn-fastapi:python3.10

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl
COPY src/prediction $APP_HOME/src/prediction

RUN pip3 install --upgrade pip
RUN pip3 install --no-cache-dir -r $APP_HOME/src/prediction/requirements.txt

CMD ["sh", "-c", "uvicorn src.prediction.main:app --host 0.0.0.0 --port $AIP_HTTP_PORT"]




In [26]:
with open(f'{DOCKERNAME_02_PRED}', 'w') as f:
    f.write(dockerfile)

In [27]:
# image definitions for training
MACHINE_TYPE            ='e2-highcpu-32'
FILE_LOCATION           = "." # './src'

print(f"FILE_LOCATION : {FILE_LOCATION}")
print(f"MACHINE_TYPE  : {MACHINE_TYPE}")

FILE_LOCATION : .
MACHINE_TYPE  : e2-highcpu-32


In [28]:
! gcloud builds submit --config ./cloudbuild.yaml \
    --substitutions _DOCKERNAME=$DOCKERNAME_02_PRED,_IMAGE_URI=$IMAGE_URI_02_PRED,_FILE_LOCATION=$FILE_LOCATION \
    --timeout=2h \
    --machine-type=$MACHINE_TYPE

Creating temporary tarball archive of 29 file(s) totalling 228.7 KiB before compression.
Uploading tarball of [.] to [gs://hybrid-vertex_cloudbuild/source/1699535196.805747-cfca63bfd02b4b7da5e5fbf71b92257e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/hybrid-vertex/locations/global/builds/5d24dcb3-4073-4a29-9cb5-7ee5e3c1526f].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/5d24dcb3-4073-4a29-9cb5-7ee5e3c1526f?project=934903580331 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "5d24dcb3-4073-4a29-9cb5-7ee5e3c1526f"

FETCHSOURCE
Fetching storage object: gs://hybrid-vertex_cloudbuild/source/1699535196.805747-cfca63bfd02b4b7da5e5fbf71b92257e.tgz#1699535197186182
Copying gs://hybrid-vertex_cloudbuild/source/1699535196.805747-cfca63bfd02b4b7da5e5fbf71b92257e.tgz#1699535197186182...
/ [1 files][ 52.6 KiB/ 52.6 KiB]                                                
Operation completed over 1 objects/52